In [1]:

import psycopg2 as pg2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, confusion_matrix, precision_score, recall_score, accuracy_score, roc_curve
import matplotlib.pyplot as plt
import numpy as np
import pickle
from datetime import datetime, date, timedelta
import importlib

In [2]:
def get_list_of_dates():

    sdate = date(2020, 2, 15)   # start date
    edate = date(2020, 2, 24)   # end date

    delta = edate - sdate       # as timedelta

    return [pd.Timestamp(sdate + timedelta(days=i)) for i in range(delta.days + 1)]
        
get_list_of_dates()

[Timestamp('2020-02-15 00:00:00'),
 Timestamp('2020-02-16 00:00:00'),
 Timestamp('2020-02-17 00:00:00'),
 Timestamp('2020-02-18 00:00:00'),
 Timestamp('2020-02-19 00:00:00'),
 Timestamp('2020-02-20 00:00:00'),
 Timestamp('2020-02-21 00:00:00'),
 Timestamp('2020-02-22 00:00:00'),
 Timestamp('2020-02-23 00:00:00'),
 Timestamp('2020-02-24 00:00:00')]

In [45]:
print ("Connecting to SQL")
conn = pg2.connect(user='postgres',  dbname='penny', host='localhost', port='5432', password='password')

print ("Reading Dataset")
df = pd.read_sql ("""Select *  from auction_full where auctiontime > '2020-01-21' order by auctiontime""", conn)

print ("Splitting into Train/Test Sets")
#df = df.sort_values("auctiontime")
y = df['is_winner']
X = df

Connecting to SQL
Reading Dataset
Splitting into Train/Test Sets


In [38]:
def print_profit_threshold (y_test, probs, X_test):
    for thresh in np.arange(.4, 1.1, .05): # threshes:
        y_pred = probs >= thresh
        cm = confusion_matrix(y_test, y_pred)
        print("*****************************")
        print("thresh:", thresh)
        acc = accuracy_score(y_test, y_pred) * 100
        prec = precision_score(y_test, y_pred) *100
        rec = recall_score(y_test, y_pred) *100
        print (f"Accuracy: {acc:.2f} Precision: {prec:.2f}  Recall:{rec:.2f} ")
        print(cm)
        true_positive_mask = (y_pred==True)&(y_test==True)
        print(true_positive_mask.shape, X_test.shape)
        profit = sum(X_test.cashvalue[true_positive_mask])-sum(y_pred)*.40 - sum(X_test.fee[true_positive_mask]) - sum(X_test.bid[true_positive_mask])/100
        profit_per_bid = profit/sum(y_pred)
        print(f"profit: {profit:.2f}")
        print(f"profit per bid: {profit_per_bid:.2f}")
        winners = X_test[true_positive_mask].groupby("cashvalue").count()
        print(winners.is_winner)
        print("*****************************")
        print("")

In [43]:
import importlib
import BuildModel
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
importlib.reload(BuildModel)


def create_model():
    # Define a Keras model
    model = Sequential()

    # Add a Dense layer that uses the sigmoid function
    model.add(Dense(units=30, init = 'uniform', activation='relu', input_dim=73))
    model.add(Dense(units=30, init = 'uniform', activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


date_list = get_list_of_dates()

#for i in range(1, len(date_list)):

train_cut_off = '2020-2-17' #date_list[i-1]
test_cut_off = '2020-03-17' #date_list[i]
print (train_cut_off, test_cut_off)
X_train = X[X.auctiontime <= train_cut_off]
y_train = y[X.auctiontime <= train_cut_off]
X_test = X[(X.auctiontime > train_cut_off)&(X.auctiontime <= test_cut_off)]
y_test = y[(X.auctiontime > train_cut_off)&(X.auctiontime <= test_cut_off)]
#X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)#, random_state=0) 



clf = KerasClassifier(build_fn=create_model,epochs=5,batch_size=10)

    
print ("Fitting Model")
pm = BuildModel.PennyModel (clf)  #(RandomForestClassifier(n_estimators=200))
pm.fit(X_train, y_train)

2020-2-17 2020-03-17
Fitting Model
2. Transforming data
4. Fitting model


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=30, activation="relu", input_dim=73, kernel_initializer="uniform")`
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=30, activation="relu", kernel_initializer="uniform")`
  app.launch_new_instance()


Epoch 1/5
6628/6628 [==============================] - 1s 102us/step - loss: 0.5996 - accuracy: 0.6818
Epoch 2/5
6628/6628 [==============================] - 1s 81us/step - loss: 0.5659 - accuracy: 0.7100
Epoch 3/5
6628/6628 [==============================] - 1s 82us/step - loss: 0.5540 - accuracy: 0.7213
Epoch 4/5
6628/6628 [==============================] - 1s 82us/step - loss: 0.5461 - accuracy: 0.7247
Epoch 5/5
6628/6628 [==============================] - 1s 82us/step - loss: 0.5392 - accuracy: 0.7317


In [44]:
    print ("Calculating predictions")
    probs = pm.predict_proba(X_test)[:,1]
    threshes = np.unique(probs)
    threshes = threshes[threshes > .4]

    print_profit_threshold(y_test, probs, X_test)

Calculating predictions
2. Transforming data
*****************************
thresh: 0.4
Accuracy: 62.03 Precision: 0.99  Recall:79.59 
[[119692  73535]
 [   189    737]]
(194153,) (194153, 82)
profit: -11911.51
profit per bid: -0.16
cashvalue
6      118
10     159
15      11
20      94
21      10
25       5
29       2
33      97
37      97
40      34
42      10
58      18
62      40
64      28
87       3
100     10
112      1
Name: is_winner, dtype: int64
*****************************

*****************************
thresh: 0.45
Accuracy: 67.94 Precision: 1.10  Recall:74.51 
[[131216  62011]
 [   236    690]]
(194153,) (194153, 82)
profit: -9237.67
profit per bid: -0.15
cashvalue
6      117
10     158
15      10
20      92
21      10
25       5
29       1
33      90
37      84
40      31
42       8
58      15
62      36
64      24
87       2
100      6
112      1
Name: is_winner, dtype: int64
*****************************

*****************************
thresh: 0.5
Accuracy: 73.34 Precisi

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(194153, 82)


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


profit: 0.00
profit per bid: nan
Series([], Name: is_winner, dtype: int64)
*****************************

*****************************
thresh: 1.0499999999999998
Accuracy: 99.52 Precision: 0.00  Recall:0.00 
[[193227      0]
 [   926      0]]
(194153,) (194153, 82)
profit: 0.00
profit per bid: nan
Series([], Name: is_winner, dtype: int64)
*****************************



In [15]:
X_train.shape

(1907331, 82)

In [16]:
pm.predict_proba(X_test)

2. Transforming data


array([[0.44 , 0.56 ],
       [0.44 , 0.56 ],
       [0.42 , 0.58 ],
       ...,
       [0.455, 0.545],
       [0.44 , 0.56 ],
       [0.415, 0.585]])

In [1]:

print ("Plotting features")
# Plot the feature importance
feat_scores = pm.get_feature_scores()
feat_scores = feat_scores.sort_values()

fig, ax = plt.subplots()
ax = feat_scores.plot(kind='barh', 
                      figsize=(10,8),
                      color='b')
ax.set_title('Average Gini Importance')
ax.set_xlabel('Average contribution to information gain')



Plotting features


NameError: name 'pm' is not defined

In [157]:
X_test[(probs > .6)].groupby("cardtype").is_winner.count().head(50)

cardtype
Amazon        13797
Home Depot     1369
None          14946
Shell          6251
Target         3643
Walmart        5528
Name: is_winner, dtype: int64

In [158]:
X_test["prob"] = probs > .6

/home/pkgalea/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [165]:
X_test.groupby(["is_bidomatic0","prob"]).mean()

auctionid  is_winner  cashvalue  cardvalue       fee  \
is_bidomatic0 prob                                                             
False         False  5.579744e+08   0.004597  63.435255  37.202811  1.276415   
              True   5.314351e+08   0.016228  41.722465  24.174205  0.943895   
True          False  5.565080e+08   0.001949  63.800840  39.593450  1.389779   
              True   5.303516e+08   0.010489  37.161703  23.826247  0.945472   

                      bidvalue  limited_allowed  is_locked         bid  \
is_bidomatic0 prob                                                       
False         False  65.581111         0.309497   0.294496  420.056325   
              True   43.870651         0.277433   0.221974  212.985805   
True          False  60.518476         0.249776   0.213906  346.425474   
              True   33.338640         0.211151   0.300181  239.660111   

                     is_bidomatic  ...  bom_bids_so_far3  bom_streak3  \
is_bidomatic0 prob                 ...                                  
False         False      0.628294  ...         28.679839    15.617112   
              True       0.720636  ...         14.373425     7.421713   
True          False      0.805810  ...         25.001725    13.043358   
              True       0.830430  ...         27.252327    15.954209   

                     perc_to_bin3  p_prev_auction_count3  prev_overbid3  \
is_bidomatic0 prob                                                        
False         False      0.236595             115.474403       0.069192   
              True       0.214227             107.473748       0.051186   
True          False      0.197456             110.687271       0.059939   
              True       0.348683             104.070326       0.053110   

                     prev_giveup_one3  prev_give_before_six3  prev_wins3  \
is_bidomatic0 prob                                                         
False         False          0.248195               0.464856    0.140230   
              True           0.270622               0.514245    0.120976   
True          False          0.259120               0.485300    0.129384   
              True           0.269415               0.510400    0.118611   

                     prev_bids3  prev_bom_bids3  
is_bidomatic0 prob                               
False         False   37.159819       28.527811  
              True    26.834702       20.325800  
True          False   33.318953       25.748146  
              True    26.303400       20.024208  

[4 rows x 67 columns]

In [148]:
X_test.groupby("prev_wins_bin").is_winner.mean()

prev_wins_bin
(-0.001, 0.05]    0.008728
(0.05, 0.1]       0.006418
(0.1, 0.15]       0.005427
(0.15, 0.2]       0.003929
(0.2, 0.25]       0.002835
(0.25, 0.3]       0.002669
(0.3, 0.35]       0.002410
(0.35, 0.4]       0.002361
(0.4, 0.45]       0.001622
(0.45, 0.5]       0.003214
(0.5, 0.55]       0.001992
(0.55, 0.6]       0.002457
(0.6, 0.65]       0.000958
(0.65, 0.7]       0.002618
(0.7, 0.75]       0.000000
(0.75, 0.8]            NaN
(0.8, 0.85]            NaN
(0.85, 0.9]            NaN
(0.9, 0.95]            NaN
(0.95, 1.0]       0.004425
Name: is_winner, dtype: float64

In [9]:
def shuffle_column(X, feature_index):
    ''' 
    Parameters
    ----------
    X: numpy array
    feature_index: int
    
    Returns
    -------
    X_new: numpy array
    
    Returns a new array identical to X but
    with all the values in column feature_index
    shuffled
    '''   
    
    X_new = X.copy()
    X_new[feature_index] = np.random.permutation(X_new[feature_index])
    return X_new    

def permutation_importance(model, X_test, y_test, scorer=accuracy_score):
    ''' Calculates permutation feature importance for a fitted model
    
    Parameters
    ----------
    model: anything with a predict() method
    X_test, y_test: numpy arrays of data
        unseen by model
    scorer: function. Should be a "higher is better" scoring function,
        meaning that if you want to use an error metric, you should
        multiply it by -1 first.
        ex: >> neg_mse = lambda y1, y2: -mean_squared_error(y1, y2)
            >> permutation_importance(mod, X, y, scorer=neg_mse)
    
    Returns
    -------
    feat_importances: numpy array of permutation importance
        for each feature
    
    '''
    
    feat_importances = np.zeros(X_test.shape[1])
    test_score = scorer(model.predict(X_test), y_test)
    for i in range(X_test.shape[1]):
        X_test_shuffled = shuffle_column(X_test, X_test.columns[i])
        test_score_permuted = scorer(y_test, model.predict(X_test_shuffled))
        print(test_score, test_score_permuted)
        feat_importances[i] = test_score - test_score_permuted
    return feat_importances

In [10]:
X_test.columns

Index(['auctionid', 'is_winner', 'cardtype', 'cashvalue', 'cardvalue', 'fee',
       'bidvalue', 'limited_allowed', 'is_locked', 'auctiontime', 'bid',
       'is_bidomatic', 'bids_so_far', 'username', 'prevusers', 'giveup',
       'eventual_bids', 'eventual_win_price', 'debut', 'bom_streak',
       'bom_bids_so_far', 'perc_to_bin', 'username0', 'distance0',
       'is_bidomatic0', 'bids_so_far0', 'bom_bids_so_far0', 'bom_streak0',
       'perc_to_bin0', 'prev_is_new_user0', 'prev_auction_count0',
       'prev_overbid0', 'prev_giveup_one0', 'prev_give_before_six0',
       'prev_wins0', 'prev_bids0', 'prev_bom_bids0', 'username1', 'distance1',
       'is_bidomatic1', 'bids_so_far1', 'bom_bids_so_far1', 'bom_streak1',
       'perc_to_bin1', 'prev_is_new_user1', 'prev_auction_count1',
       'prev_overbid1', 'prev_giveup_one1', 'prev_give_before_six1',
       'prev_wins1', 'prev_bids1', 'prev_bom_bids1', 'username2', 'distance2',
       'is_bidomatic2', 'bids_so_far2', 'bom_bids_so_far2', 

In [11]:
pis = permutation_importance(pm, X_test, y_test)

2. Transforming data
2. Transforming data
0.6916629721113856 0.6916629721113856
2. Transforming data
0.6916629721113856 0.6916629721113856
2. Transforming data
0.6916629721113856 0.6894654314376906
2. Transforming data
0.6916629721113856 0.706863428125066
2. Transforming data
0.6916629721113856 0.6916629721113856
2. Transforming data
0.6916629721113856 0.6916629721113856
2. Transforming data
0.6916629721113856 0.6974725403378262
2. Transforming data
0.6916629721113856 0.6924997648919367
2. Transforming data
0.6916629721113856 0.6922944051957922
2. Transforming data
0.6916629721113856 0.6910852780128859
2. Transforming data
0.6916629721113856 0.7024069307937824
2. Transforming data
0.6916629721113856 0.6909547690471118
2. Transforming data
0.6916629721113856 0.6916629721113856
2. Transforming data
0.6916629721113856 0.6916629721113856
2. Transforming data
0.6916629721113856 0.700437780810192
2. Transforming data
0.6916629721113856 0.6916629721113856
2. Transforming data
0.69166297211138

In [12]:
for c, pi in sorted(zip(X_test.columns, pis), key=lambda x: abs(x[1])):
    print(c, pi)
    
key=lambda x: x[1]

auctionid 0.0
is_winner 0.0
cardvalue 0.0
fee 0.0
bids_so_far 0.0
username 0.0
giveup 0.0
eventual_bids 0.0
eventual_win_price 0.0
debut 0.0
bom_streak 0.0
bom_bids_so_far 0.0
perc_to_bin 0.0
username0 0.0
distance0 0.0
bom_bids_so_far0 0.0
bom_streak0 0.0
prev_is_new_user0 0.0
username1 0.0
bom_bids_so_far1 0.0
bom_streak1 0.0
prev_is_new_user1 0.0
username2 0.0
bom_bids_so_far2 0.0
bom_streak2 0.0
prev_is_new_user2 0.0
username3 0.0
bom_bids_so_far3 0.0
bom_streak3 0.0
prev_is_new_user3 0.0
perc_to_bin3 0.00021687519312452164
distance3 -0.0003032414204748868
prev_auction_count1 0.0005086011166194337
prev_overbid3 0.00053547060957293
prev_bids2 0.0005431476075595798
auctiontime 0.0005776940984997259
is_locked -0.0006314330844066074
is_bidomatic3 0.0007043645652804464
is_bidomatic 0.0007082030642737713
limited_allowed -0.0008367927805510433
prev_auction_count3 0.0009365937543783787
prev_giveup_one2 0.001015282983742094
is_bidomatic2 0.0014874183599246082
is_bidomatic0 0.001552672842811

In [58]:
np.random.permutation(X_new["cardtype"])

array(['Amazon', 'Walmart', 'None', ..., 'Target', 'Target', 'Amazon'],
      dtype=object)

In [71]:
conn.close()